In [3]:
# הסרה של גרסאות קיימות
#!pip uninstall -y xgboost scikit-learn autoviz

# התקנת גרסאות תואמות:
#!pip install xgboost==1.6.2 scikit-learn==1.4.2 autoviz

# לאחר ההתקנה – חובה לאתחל את הסשן:
# import os, sys
# os.kill(os.getpid(), 9)

import xgboost, sklearn
import inspect

print("xgboost:", xgboost.__version__)
print("sklearn:", sklearn.__version__)
print("xgboost location:", inspect.getfile(xgboost))



xgboost: 3.1.0
sklearn: 1.6.1
xgboost location: /usr/local/lib/python3.12/dist-packages/xgboost/__init__.py


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
from google.colab import files
import pickle
import os
from scipy import stats
from scipy.stats import zscore

!pip install missingno
import missingno as msno

!pip install fancyimpute
import fancyimpute

#!pip install --upgrade autoviz
#from autoviz.AutoViz_Class import AutoViz_Class
#!pip install -U "xgboost>=2.0.3"
# מומלץ לאשר ריסטרט: Runtime → Restart runtime
import sklearn, xgboost
print("sklearn:", sklearn.__version__, "xgboost:", xgboost.__version__)

warnings.filterwarnings("ignore")
%matplotlib inline

sklearn: 1.6.1 xgboost: 3.1.0


In [ ]:
np.random.seed(31071967)

In [ ]:
download_pickels = True

In [5]:
import os, pathlib, sys
!mkdir -p pickles
pickle_file = '/content/pickles/load_default_feature_eng.pkl'

if not os.path.exists(pickle_file):
# Upload file
  uploaded = files.upload()

# Read pickle into DataFrame
df = pd.read_pickle(pickle_file)

df.head(10)

,id,loan_amnt,installment,purpose,addr_state,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,last_credit_pull_d_month,months_between_las_p_to_issue,credit_age_months,payment_to_income,underfunded_amt,platform_share_amt,term_num,amount_x_rate,rate_x_term,repay_fail
0,545583,2500,85.0,9,22,0.0,5.0,7.0,0.0,6.889591,...,6.0,36,59,102.991421,0,0,36,349.5,5.0328,0
1,532101,5000,176.0,2,33,0.0,1.0,7.0,0.0,9.840228,...,3.0,17,194,192.256756,0,0,36,797.5,5.7420,1
2,877788,7000,226.0,9,42,3.0,3.0,7.0,0.0,8.092545,...,3.0,30,162,248.967844,0,0,36,693.7,3.5676,0
3,875406,2000,60.0,2,33,0.0,0.0,7.0,0.0,7.547643,...,6.0,29,440,69.841981,0,0,36,108.4,1.9512,0
4,506439,3600,117.0,9,1,0.0,4.0,8.0,0.0,7.278170,...,6.0,37,144,128.923034,0,0,36,369.0,3.6900,0
5,981465,8000,243.0,9,3,0.0,0.0,12.0,0.0,8.338784,...,4.0,30,183,258.944816,0,0,36,482.4,2.1708,0
6,749050,6000,187.0,2,24,0.0,0.0,5.0,0.0,8.676758,...,5.0,36,91,214.467333,0,0,36,449.4,2.6964,0
7,1016373,25600,599.0,2,33,0.0,1.0,16.0,0.0,10.404929,...,6.0,29,337,632.628741,127,127,60,3653.12,8.5620,0
8,786870,19750,559.0,2,24,0.0,0.0,15.0,0.0,9.977899,...,2.0,24,231,599.391193,0,0,60,4585.95,13.9320,0
9,1019261,6250,224.0,9,34,0.0,0.0,2.0,1.0,6.405507,...,9.0,29,145,262.500115,0,0,36,1079.375,6.2172,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38478 entries, 0 to 38477
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             38478 non-null  int64  
 1   loan_amnt                      38478 non-null  Int64  
 2   installment                    38478 non-null  float64
 3   purpose                        38478 non-null  int8   
 4   addr_state                     38478 non-null  int8   
 5   delinq_2yrs                    38478 non-null  float64
 6   inq_last_6mths                 38478 non-null  float64
 7   open_acc                       38478 non-null  float64
 8   pub_rec                        38478 non-null  float64
 9   revol_bal                      38478 non-null  float64
 10  revol_util                     38478 non-null  float64
 11  total_acc                      38478 non-null  float64
 12  total_pymnt                    38478 non-null 

In [6]:
df['amount_x_rate'] = pd.to_numeric(df['amount_x_rate'], errors='coerce').astype('float32')

In [7]:
from sklearn.model_selection import train_test_split
df_model = df.copy()
# Assuming df_model is already defined and contains the necessary features and target variable

# Define the target and feature set
X = df_model.drop(columns=['repay_fail'])
y = df_model['repay_fail']

# Total number of samples
total_samples = len(df_model)

# First split: training and temporary (test + dev)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

# Second split: testing and development (dev)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Calculate percentages
train_percent = (X_train.shape[0] / total_samples) * 100
dev_percent = (X_dev.shape[0] / total_samples) * 100
test_percent = (X_test.shape[0] / total_samples) * 100

# Output the sizes of each set in percentages
print(f"Training set size: {train_percent:.2f}%")
print(f"Validation (Dev) set size: {dev_percent:.2f}%")
print(f"Testing set size: {test_percent:.2f}%")

Training set size: 70.00%
Validation (Dev) set size: 15.00%
Testing set size: 15.00%


In [8]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier #Regressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost
from xgboost import XGBClassifier

In [9]:
print(xgboost.__version__)

3.1.0


In [10]:
#!pip install -U "xgboost>=2.0.3"
# מומלץ לאשר ריסטרט: Runtime → Restart runtime
import sklearn, xgboost
print("sklearn:", sklearn.__version__, "xgboost:", xgboost.__version__)


sklearn: 1.6.1 xgboost: 3.1.0


In [ ]:
# !pip install -U xgboost

# # If you need to be explicit/pinned:
# !pip install -U "scikit-learn==1.6.1" "xgboost>=2.1.4"


In [11]:
import sys, xgboost, sklearn, inspect
print("Python:", sys.executable)
print("xgboost version:", xgboost.__version__)
print("xgboost location:", inspect.getfile(xgboost))
print("sklearn version:", sklearn.__version__)

Python: /usr/bin/python3
xgboost version: 3.1.0
xgboost location: /usr/local/lib/python3.12/dist-packages/xgboost/__init__.py
sklearn version: 1.6.1


In [12]:
# --- classification function ---
def classificationMetrics(y, yhat):
    prf1 = metrics.precision_recall_fscore_support(y,yhat)
    res = {'Accuracy': metrics.accuracy_score(y,yhat),
           'Precision':prf1[0][1],
           'Recall': prf1[1][1],
           'f1-score': prf1[2][1],
           'Log-loss': metrics.log_loss(y,yhat),
           'AUC': metrics.roc_auc_score(y,yhat)
          }
    return res
# --- models dictionary ---
models = {
    "LogisticClassifier": LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "SVC": SVC(),
    "XGBClassifier": xgboost.XGBClassifier(objective="reg:squarederror")
}

# --- train, predict, and evaluate on DEV set ---
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_dev_pred = model.predict(X_dev)
    m = classificationMetrics(y_dev, y_dev_pred)
    results.append({"Model": name, **m})

# --- results table ---
df_results = pd.DataFrame(results)
df_results

,Model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,LogisticClassifier,0.850312,0.560000,0.016148,0.031390,5.395308,0.506953
1,DecisionTreeClassifier,0.946639,0.819429,0.826990,0.823192,1.923327,0.897389
2,RandomForestClassifier,0.955994,0.970814,0.728950,0.832675,1.586121,0.862538
3,AdaBoostClassifier,0.926022,0.923077,0.553633,0.692141,2.666431,0.772739
4,GradientBoostingClassifier,0.955301,0.955157,0.737024,0.832031,1.611099,0.865454
5,SVC,0.849792,0.000000,0.000000,0.000000,5.414041,0.500000
6,XGBClassifier,0.975225,0.964103,0.867359,0.913175,0.892973,0.930825


The table presents a comparison of classification models using standard evaluation metrics.

General rule of thumb:

Higher is better: Accuracy, Precision, Recall, F1-score, AUC

Lower is better: Log-loss

Which model performs best?

The XGBClassifier outperforms all other models across almost all metrics:

Accuracy: 0.975 (highest)

Precision: 0.964

Recall: 0.867 (highest)

F1-score: 0.913 (highest)

Log-loss: 0.892 (lowest)

AUC: 0.930 (highest)

This means that XGBClassifier:

Detects the highest number of positive cases (high Recall),

Maintains a low number of False Positives (high Precision),

Is well-balanced between Precision and Recall (high F1-score),

Produces well-calibrated probability outputs (low Log-loss),

And has excellent overall discrimination ability between the classes (high AUC).

השוואת מודלי סיווג לפי מדדים סטנדרטיים.
כלל אצבע:

גבוה יותר טוב: Accuracy, Precision, Recall, F1-score, AUC

נמוך יותר טוב: Log-loss

מי הכי טוב כאן?

XGBClassifier מוביל כמעט בכל המדדים:

Accuracy 0.979 (הגבוה ביותר)

Precision 0.9794

Recall 0.8778 (הגבוה ביותר)

F1 0.9258 (הגבוה ביותר)

Log-loss 0.7619 (הנמוך ביותר)

AUC 0.9373 (הגבוה ביותר)

כלומר הוא גם מזהה הכי הרבה חיוביים (Recall), גם שומר על מעט False Positives (Precision), גם מאוזן (F1), גם נותן הסתברויות טובות (Log-loss נמוך), וגם בעל יכולת הבחנה כללית מצוינת (AUC).

**Finetuning Evaluation**

In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

xgb = XGBClassifier(
    random_state=42,
    tree_method="hist",   # fast default
    n_jobs=-1,
    eval_metric="auc"     # good default for binary clsf.
)

param_dist = {
    "n_estimators": [200, 400, 800],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 4, 5, 6],
    "subsample": [0.7, 0.85, 1.0],
    "colsample_bytree": [0.7, 0.85, 1.0],
    "min_child_weight": [1, 3, 5, 10],
    "gamma": [0, 0.1, 0.2],
    "reg_alpha": [0, 1e-3, 1e-2, 1e-1],
    "reg_lambda": [0.5, 1.0, 2.0],
}

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=2,
    random_state=42
)

random_search.fit(X_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
{'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 0.001, 'n_estimators': 400, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 1.0}
0.9907678257485625


In [14]:
from sklearn.metrics import roc_auc_score, average_precision_score
random_search.fit(X_train, y_train)
print("Best params:", random_search.best_params_)
print("Best CV ROC-AUC:", random_search.best_score_)

best_model = random_search.best_estimator_

# TEST
y_proba = best_model.predict_proba(X_test)[:, 1]
print("Test ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Test PR-AUC :", average_precision_score(y_test, y_proba))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best params: {'subsample': 1.0, 'reg_lambda': 1.0, 'reg_alpha': 0.001, 'n_estimators': 400, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 1.0}
Best CV ROC-AUC: 0.9907678257485625
Test ROC-AUC: 0.9895664838464644
Test PR-AUC : 0.9760698226017348


***Conclusion*** using finetuning parameters

In [17]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    log_loss, roc_auc_score, average_precision_score
)

# ----- 1) Best hyperparameters you found -----
best_params = {
    "subsample": 1.0,
    "reg_lambda": 1.0,
    "reg_alpha": 0.001,
    "n_estimators": 400,
    "min_child_weight": 1,
    "max_depth": 6,
    "learning_rate": 0.1,
    "gamma": 0.1,
    "colsample_bytree": 1.0,
}

# ----- 2) Fit XGBoost with those params -----
xgb_best = XGBClassifier(
    random_state=42,
    tree_method="hist",      # fast CPU histogram algorithm
    n_jobs=-1,
    eval_metric="logloss",   # convenient for computing log-loss; AUC computed separately
    **best_params
)

xgb_best.fit(X_train, y_train)

# ----- 3) Predict on the test set -----
y_proba = xgb_best.predict_proba(X_test)[:, 1]   # class-1 probabilities
y_pred  = (y_proba >= 0.5).astype(int)           # decision threshold (can be tuned)

# ----- 4) Compute all metrics -----
acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec  = recall_score(y_test, y_pred, zero_division=0)
f1   = f1_score(y_test, y_pred, zero_division=0)
ll   = log_loss(y_test, y_proba)
roc  = roc_auc_score(y_test, y_proba)
pra  = average_precision_score(y_test, y_proba)  # PR-AUC (Average Precision)

# ----- 5) Assemble the results into an English table -----
rows = [
    {
        "Metric": "Accuracy",
        "What it means": "Share of correct predictions over all samples",
        "How it’s computed": "(TP + TN) / (TP + TN + FP + FN)",
        "Good value": "High (close to 1). Can be misleading with class imbalance.",
        "Your model": f"{acc:.4f}",
    },
    {
        "Metric": "Precision",
        "What it means": "Of all predicted positives, how many are truly positive?",
        "How it’s computed": "TP / (TP + FP)",
        "Good value": "High (≈0.8–1). Low means many false positives.",
        "Your model": f"{prec:.4f}",
    },
    {
        "Metric": "Recall (Sensitivity)",
        "What it means": "Of all true positives, how many did the model catch?",
        "How it’s computed": "TP / (TP + FN)",
        "Good value": "High (≈0.8–1). Low means many missed positives.",
        "Your model": f"{rec:.4f}",
    },
    {
        "Metric": "F1-score",
        "What it means": "Harmonic mean of Precision and Recall (balance between them)",
        "How it’s computed": "2 × (P × R) / (P + R)",
        "Good value": "Closer to 1 is better, especially with class imbalance.",
        "Your model": f"{f1:.4f}",
    },
    {
        "Metric": "Log-loss",
        "What it means": "Penalty for wrong/confident probability estimates",
        "How it’s computed": "Cross-entropy on predicted probabilities",
        "Good value": "Lower is better (0 is ideal).",
        "Your model": f"{ll:.6f}",
    },
    {
        "Metric": "ROC-AUC",
        "What it means": "Area under ROC; separation ability across all thresholds",
        "How it’s computed": "Based on TPR vs. FPR over thresholds",
        "Good value": "Close to 1 (0.5 is random).",
        "Your model": f"{roc:.4f}",
    },
    {
        "Metric": "PR-AUC (Average Precision)",
        "What it means": "Area under Precision–Recall; sensitive to class imbalance",
        "How it’s computed": "Weighted average of Precision over Recall",
        "Good value": "Higher is better; useful with rare positives.",
        "Your model": f"{pra:.4f}",
    },
]

metrics_table_en = pd.DataFrame(
    rows,
    columns=["Metric", "What it means", "How it’s computed", "Good value", "Your model"]
)

metrics_table_en


,Metric,What it means,How it’s computed,Good value,Your model
0,Accuracy,Share of correct predictions over all samples,(TP + TN) / (TP + TN + FP + FN),High (close to 1). Can be misleading with clas...,0.9801
1,Precision,"Of all predicted positives, how many are truly...",TP / (TP + FP),High (≈0.8–1). Low means many false positives.,0.9799
2,Recall (Sensitivity),"Of all true positives, how many did the model ...",TP / (TP + FN),High (≈0.8–1). Low means many missed positives.,0.8874
3,F1-score,Harmonic mean of Precision and Recall (balance...,2 × (P × R) / (P + R),"Closer to 1 is better, especially with class i...",0.9313
4,Log-loss,Penalty for wrong/confident probability estimates,Cross-entropy on predicted probabilities,Lower is better (0 is ideal).,0.064837
5,ROC-AUC,Area under ROC; separation ability across all ...,Based on TPR vs. FPR over thresholds,Close to 1 (0.5 is random).,0.9896
6,PR-AUC (Average Precision),Area under Precision–Recall; sensitive to clas...,Weighted average of Precision over Recall,Higher is better; useful with rare positives.,0.9761
